In [7]:
import sqlite3
import pandas as pd

con = sqlite3.connect('users.sb')
cursor = con.cursor()

def init():
	cursor.execute('''
		CREATE TABLE if not exists users(
			name varchar(128),
			lastname varchar(128),                          
			age integer,
			salary integer,
			deleted_flg integer default 0,
			start_dttm datetime default current_timestamp,
			end_dttm datetime default (datetime('2999-12-31 23:59:59')))
	''')
	cursor.execute('''
			CREATE VIEW if not exists v_users as
			select
				id,
				name,
				lastname,
				age,
				salary
			from users
			where deleted_flg = 0 and
			current_timestamp between start_dttm and end_dttm;
		''')

def showTable(table):
	cursor.execute(f'select * from {table}')
	for row in cursor.fetchall():
		print(row)

def deleteRow(con, name, lastname):
	cursor = con.cursor()
	cursor.execute('''
		UPDATE users
		set deleted_flg = 1, end_dttm = datetime('now', '-1 second')
		where name = ? and lastname = ? and end_dttm = datetime('2999-12-31 23:59:59')
		''', [name, lastname])
	con.commit()
 
def showDel(table):
	cursor.execute(f'select * from {table} where deleted_flg=1')
	for row in cursor.fetchall():
		print(row)

def addRow(con, name, lastname, age, salary):
  cursor = con.cursor()
  cursor.execute('''
		UPDATE users
		set end_dttm = datetime('now', '-1 second')
		where name = ? and lastname = ?
		and end_dttm = datetime('2999-12-31 23:59:59')
		''', [name, lastname])
  cursor.execute('''
		  INSERT INTO users (name, lastname, age, salary) values(?, ?, ?, ?)
	  ''', [name, lastname, age, salary])
  con.commit()


def getRows(date):
	if date:
		query = f"select * from users where datetime('{date}') between start_dttm and end_dttm"
		results = pd.read_sql_query(query, con=con, parse_dates=True)
		results.to_csv("output.csv", index=False)
	else:
		showTable('users')
	con.commit()

In [8]:
init()

In [9]:
addRow(con, 'name1', 'lastname2', 15, 300)

In [10]:
addRow(con, 'name1', 'lastname2', 15, 300)
addRow(con, 'Иван', 'Иванов', 21, 30000)    
addRow(con, 'Петр', 'Петров', 35, 25000)
addRow(con, 'Вадим', 'Сидоров', 27, 18000)
addRow(con, 'Борис', 'Кузнецов', 56, 73000)

In [11]:
showTable('users')

('name1', 'lastname2', 15, 300, 0, '2021-06-07 14:42:27', '2021-06-07 14:43:28')
('name1', 'lastname2', 15, 300, 0, '2021-06-07 14:43:29', '2999-12-31 23:59:59')
('Иван', 'Иванов', 21, 30000, 0, '2021-06-07 14:43:29', '2999-12-31 23:59:59')
('Петр', 'Петров', 35, 25000, 0, '2021-06-07 14:43:29', '2999-12-31 23:59:59')
('Вадим', 'Сидоров', 27, 18000, 0, '2021-06-07 14:43:29', '2999-12-31 23:59:59')
('Борис', 'Кузнецов', 56, 73000, 0, '2021-06-07 14:43:29', '2999-12-31 23:59:59')


In [12]:
deleteRow(con, 'name1', 'lastname2')

In [13]:
showTable('users')

('name1', 'lastname2', 15, 300, 0, '2021-06-07 14:42:27', '2021-06-07 14:43:28')
('name1', 'lastname2', 15, 300, 1, '2021-06-07 14:43:29', '2021-06-07 14:44:46')
('Иван', 'Иванов', 21, 30000, 0, '2021-06-07 14:43:29', '2999-12-31 23:59:59')
('Петр', 'Петров', 35, 25000, 0, '2021-06-07 14:43:29', '2999-12-31 23:59:59')
('Вадим', 'Сидоров', 27, 18000, 0, '2021-06-07 14:43:29', '2999-12-31 23:59:59')
('Борис', 'Кузнецов', 56, 73000, 0, '2021-06-07 14:43:29', '2999-12-31 23:59:59')


In [14]:
showDel('users')

('name1', 'lastname2', 15, 300, 1, '2021-06-07 14:43:29', '2021-06-07 14:44:46')


In [15]:
getRows(0)

('name1', 'lastname2', 15, 300, 0, '2021-06-07 14:42:27', '2021-06-07 14:43:28')
('name1', 'lastname2', 15, 300, 1, '2021-06-07 14:43:29', '2021-06-07 14:44:46')
('Иван', 'Иванов', 21, 30000, 0, '2021-06-07 14:43:29', '2999-12-31 23:59:59')
('Петр', 'Петров', 35, 25000, 0, '2021-06-07 14:43:29', '2999-12-31 23:59:59')
('Вадим', 'Сидоров', 27, 18000, 0, '2021-06-07 14:43:29', '2999-12-31 23:59:59')
('Борис', 'Кузнецов', 56, 73000, 0, '2021-06-07 14:43:29', '2999-12-31 23:59:59')


In [16]:
getRows('2021-06-07 14:44:50')

In [17]:
with open('/content/output.csv') as f:
  for row in f:
    print(f.read())


Иван,Иванов,21,30000,0,2021-06-07 14:43:29,2999-12-31 23:59:59
Петр,Петров,35,25000,0,2021-06-07 14:43:29,2999-12-31 23:59:59
Вадим,Сидоров,27,18000,0,2021-06-07 14:43:29,2999-12-31 23:59:59
Борис,Кузнецов,56,73000,0,2021-06-07 14:43:29,2999-12-31 23:59:59



In [18]:
getRows('2021-06-07 14:43:02')

In [19]:
with open('/content/output.csv') as f:
  for row in f:
    print(f.read())

name1,lastname2,15,300,0,2021-06-07 14:42:27,2021-06-07 14:43:28

